In [ ]:
"""

This code only extracts the 2018 flooded and jrc_perm_water raster data layers, 
as well as the vector data of image properties, from the Global Flood Database v1 (2000-2018) product. 
It does not alter the original data’s spatial resolution. For subsequent analysis, see Dataprepare-Disasters.R.

Script Functionality:
The script reads a local shapefile for the region boundary, filters flood event data, applies conditions to isolate specific flood areas, and exports the processed image to Google Drive.

Input Parameters:
Shapefile Path: File path for the shapefile containing regional boundaries.
Google Earth Engine Project Name: Project identifier required for Earth Engine initialization.

Dependencies:
Earth Engine API: pip install earthengine-api
Geopandas: pip install geopandas (for reading shapefiles)

Version Information:
Version: v1.0
Change Log: October 30, 2024.
"""
import ee
import geopandas as gpd

# Initialize Google Earth Engine
ee.Authenticate()  # Authenticate Earth Engine (requires Google account login)
ee.Initialize(project="lizeshuo")  # Initialize Earth Engine with specified project ID

# Read the local shapefile
# Use geopandas to read the shapefile and convert it to GeoJSON format
gdf = gpd.read_file("E:/ghana/Province/1/1/Ghana_Expanded_Boundaries.shp")
geometry = gdf.geometry[0]  # Assuming there is only one region in the shapefile
region = ee.Geometry.Polygon(geometry.__geo_interface__['coordinates'])  # Convert to GEE-compatible format

# Load flood dataset
dataset = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1")

# Filter dataset to only include data from 2018
flood_2018 = dataset.filterDate("2018-01-01", "2018-12-31")

# Select the 'flooded' and 'jrc_perm_water' bands
flooded_band = flood_2018.select("flooded").sum()  # Sum all 'flooded' pixels over the year
jrc_perm_water_band = flood_2018.select("jrc_perm_water").sum()  # Sum permanent water pixels over the year

# Define condition to identify pixels that are flooded but not permanently water-covered
condition = jrc_perm_water_band.eq(0).And(flooded_band.eq(1))

# Export the filtered image to Google Drive
export_filtered_image = ee.batch.Export.image.toDrive(
    image=condition,
    description='Flood_GLD_flooded_jrc_perm_water_2018',  # Task name for the export
    scale=30,  # Spatial resolution in meters
    region=region,  # Area of interest from shapefile
    maxPixels=1e10  # Set maximum pixel count for export
)

export_filtered_image.start()  # Begin the export task
# End of the script



In [ ]:
"""
This code only extracts the 2018 flooded and jrc_perm_water raster data layers, 
as well as the vector data of image properties, from the Global Flood Database v1 (2000-2018) product. 
It does not alter the original data’s spatial resolution. For subsequent analysis, see Dataprepare-Disasters.R.

Script Functionality:
The script reads a local shapefile for the region boundary, filters the flood dataset by date, extracts image properties from each flood event, and exports the properties as a CSV file to Google Drive for analysis.

Input Parameters:
Shapefile: A local shapefile containing the boundary of the target region.
Flood Event Data: MODIS Global Flood Database (Earth Engine) for analyzing flood events within 2018.

Dependencies:
Earth Engine Python API: For data access and processing. Install with pip install earthengine-api
Geopandas: To read shapefiles and convert them to GeoJSON format compatible with Earth Engine. Install with pip install geopandas

Version Information:
Version: v1.0
Change Log: October 30, 2024
"""
import ee
import geopandas as gpd

# Initialize Earth Engine
ee.Authenticate()  # Authenticate Earth Engine (requires Google account login)
ee.Initialize(project="lizeshuo")  # Initialize Earth Engine with specified project ID

# Define the ImageCollection
collection = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1")

# Read the local shapefile
# Use geopandas to read the shapefile and convert it to GeoJSON format
gdf = gpd.read_file("E:/ghana/Province/1/1/Ghana_Expanded_Boundaries.shp")
geometry = gdf.geometry[0]  # Assuming there is only one region in the shapefile
region = ee.Geometry.Polygon(geometry.__geo_interface__['coordinates'])  # Convert to GEE-compatible format

# Load the flood dataset
collection = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1")

# Filter the dataset to include only data from 2018
collection = collection.filterDate("2018-01-01", "2018-12-31")

# Define a function to extract properties of each image
def get_image_properties(image):
    return ee.Feature(None, image.toDictionary(image.propertyNames()))  # Convert image properties to Feature

# Map the function over the collection and convert all image properties to a FeatureCollection
properties_collection = ee.FeatureCollection(collection.map(get_image_properties))

# Set up an export task to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=properties_collection,
    description='Flood_GLD_image_properties_2018',  # Task description for export
    fileFormat='CSV'  # Export file format
)

# Start the export task
task.start()

print('Export task has started. Check Google Drive for the file upon completion.')
# End of the script


In [ ]:
"""

This code only extracts the 2018 flooded and jrc_perm_water raster data layers, 
as well as the vector data of image properties, from the Global Flood Database v1 (2000-2018) product. 
It does not alter the original data’s spatial resolution. For subsequent analysis, see Dataprepare-Disasters.R.

Script Functionality:
This script processes flood image property data from a CSV file, preparing it for spatial analysis by converting it to a Shapefile format. 
The script performs essential tasks, such as renaming columns for compatibility, truncating long text fields, creating point geometries from centroid coordinates, and setting the coordinate reference system to WGS84 (EPSG:4326).

Input Parameters:
A local CSV file with flood event metadata, including centroid coordinates (cent_x and cent_y) for each event.

Dependencies:
Pandas: For loading and manipulating CSV data. Install via pip install pandas
Geopandas: For handling spatial data and exporting it as a Shapefile. Install via pip install geopandas
Shapely: For creating point geometries. Installed automatically with Geopandas

Version Information:
Version: v1.0
Change Log: October 30, 2024
"""


import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Read the CSV file
file_path = 'E:/ghana/flood/flood_image_properties.csv'
data = pd.read_csv(file_path)  # Load data from CSV into a DataFrame

# Rename columns to ensure field names are under 10 characters
data.rename(columns={
    'system:index': 'sys_index',
    'dfo_centroid_x': 'cent_x',
    'dfo_centroid_y': 'cent_y',
    'system:asset_size': 'asset_size',
    'system:footprint': 'footprint',
    'system:id': 'sys_id',
    'system:time_end': 'time_end',
    'system:time_start': 'time_start',
    'system:version': 'version',
    'threshold_b1b2': 'thresh_b1b2',
    'threshold_b7': 'thresh_b7',
    'threshold_type': 'thresh_type'
}, inplace=True)

# Truncate the `footprint` text field to ensure it does not exceed 254 characters
data['footprint'] = data['footprint'].astype(str).str.slice(0, 254)

# Create geometry points using `cent_x` and `cent_y` coordinates
data['geometry'] = data.apply(lambda row: Point(row['cent_x'], row['cent_y']), axis=1)

# Drop unnecessary fields, including `.geo` and timestamp fields
data.drop(columns=['.geo', 'time_end', 'time_start'], inplace=True)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry='geometry')

# Set the coordinate reference system (WGS84 - EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Save the GeoDataFrame as a Shapefile
shapefile_path = 'F:/[2024.07]Ghana/Processed Data/Disasters/Extraction/Flood_GLD_image_properties_2018.shp'
gdf.to_file(shapefile_path, driver='ESRI Shapefile')

print(f"Conversion complete. Shapefile saved at {shapefile_path}")
# End of the script

转换完成，Shapefile已保存为 E:/ghana/flood/shp/flood_image_properties.shp


C:\Users\11693\AppData\Local\Temp\ipykernel_28996\2322547566.py:42: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(shapefile_path, driver='ESRI Shapefile')
d:\anaconda3\envs\gee\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'composite_type' to 'composite_'
  ogr_write(
d:\anaconda3\envs\gee\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'dfo_country' to 'dfo_countr'
  ogr_write(
d:\anaconda3\envs\gee\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'dfo_displaced' to 'dfo_displa'
  ogr_write(
d:\anaconda3\envs\gee\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'dfo_main_cause' to 'dfo_main_c'
  ogr_write(
d:\anaconda3\envs\gee\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'dfo_other_country' to 'dfo_other_'
  ogr_write(
d:\